In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup

In [6]:
from backend.contextbased.context_helper import ContextHelper
from backend.contextbased.context import Context
from backend.contextbased.sparql_generator import generate_sparql_from_context
from backend.history import History
import yaml

config = None
with open('config.yaml', 'r') as stream:
    try:
        config = yaml.load(stream)
        #print('Configurations loaded:\n%s' % config)
    except yaml.YAMLError as exc:
        print(exc)

history = History(config['logserver']['hostname'],
                       config['logserver']['port'],
                       config['logserver']['db'],
                       config['logserver']['collection'])

sparql_endpoint = config['imdb_sparql']['url']
rasa_endpoint = config['rasa']['url']

# instantiate context helper
ch = ContextHelper(sparql_endpoint, rasa_endpoint)

rasa = ch.rasa.rasa # ch.rasa is rasa_extractor; ch.rasa.rasa is rasa_connector

### Test 1
User: *list movies of 2016*

Bot: list of movies

User: *only the ones with Brad Pitt*

In [7]:
text1 = "list movies of 2016"
intent, intent_confidence, entities, properties, literal_objects, ranges = ch.rasa.extract_from_text(text1)

old_context = Context(entities)

query, select_columns = generate_sparql_from_context(old_context, prefixes=False)
print query

Intent: moviesOfYear
Intent confidence 1.140854
Result entities: [{u'extractor': u'ner_mitie', u'end': 19, u'start': 15, u'value': u'2016', u'entity': u'dbpprop:releaseDate'}]
[<backend.contextbased.context.Entity object at 0x1045af290>, <backend.contextbased.context.Entity object at 0x1045afc10>]
 
 SELECT  ?Film_1_dbpprop_name WHERE { 
?Film1 rdf:type dbp-owl:Film.
?Film1 dbpprop:name ?Film_1_dbpprop_name.
?Film1 ns:release ?Release1.
?Release1 rdf:type ns:Release.
?Release1 dbpprop:releaseDate 2016.
?Release1 dbpprop:releaseLocation "USA". 
 } LIMIT 10


In [8]:
text2 = "only the ones with Brad Pitt"

result = rasa.query(text2)

intent = result['intent']['name']
intent_confidence = result['intent']['confidence']
entities = result['entities']
print entities

print 'Intent: %s' % intent
print 'Intent confidence: %s' % intent_confidence
print 'Entities: %s' % ' '.join(['(%s,%s)' % (e['entity'],e['value']) for e in entities])

[{u'extractor': u'ner_mitie', u'end': 23, u'start': 19, u'value': u'Brad', u'entity': u'foaf:name'}, {u'extractor': u'ner_mitie', u'end': 28, u'start': 24, u'value': u'Pitt', u'entity': u'foaf:familyName'}]
Intent: moviesOfDirector
Intent confidence: 0.299188035848
Entities: (foaf:name,Brad) (foaf:familyName,Pitt)


In [9]:
from backend.contextbased.default_intents import *

new_context = Context(moviesOfDirector(entities, old_context))

query, select_columns = generate_sparql_from_context(new_context, prefixes=False)
print query

a
 
 SELECT  ?Film_1_dbpprop_name WHERE { 
?Film1 rdf:type dbp-owl:Film.
?Film1 dbpprop:name ?Film_1_dbpprop_name.
?Film1 ns:release ?Release1.
?Film1 dbpprop:director ?MovieDirector1.
?Release1 rdf:type ns:Release.
?Release1 dbpprop:releaseDate 2016.
?Release1 dbpprop:releaseLocation "USA".
?MovieDirector1 rdf:type dbp-owl:MovieDirector.
?MovieDirector1 foaf:name "Brad".
?MovieDirector1 foaf:familyName "Pitt". 
 } LIMIT 10


### Test 2
User: *list movies of 2016*

Bot: list of movies

User: *only horror movies*

In [11]:
text1 = "list movies of 2016"
intent, intent_confidence, entities, properties, literal_objects, ranges = ch.rasa.extract_from_text(text1)

old_context = Context(entities)

query, select_columns = generate_sparql_from_context(old_context, prefixes=False)
print query

Intent: moviesOfYear
Intent confidence 1.140854
Result entities: [{u'extractor': u'ner_mitie', u'end': 19, u'start': 15, u'value': u'2016', u'entity': u'dbpprop:releaseDate'}]
[<backend.contextbased.context.Entity object at 0x1096f1090>, <backend.contextbased.context.Entity object at 0x109759d10>]
 
 SELECT  ?Film_1_dbpprop_name WHERE { 
?Film1 rdf:type dbp-owl:Film.
?Film1 dbpprop:name ?Film_1_dbpprop_name.
?Film1 ns:release ?Release1.
?Release1 rdf:type ns:Release.
?Release1 dbpprop:releaseDate 2016.
?Release1 dbpprop:releaseLocation "USA". 
 } LIMIT 10


In [16]:
text2 = "only Horror movies"

result = rasa.query(text2)

intent = result['intent']['name']
intent_confidence = result['intent']['confidence']
entities = result['entities']
print entities

print 'Intent: %s' % intent
print 'Intent confidence: %s' % intent_confidence
print 'Entities: %s' % ' '.join(['(%s,%s)' % (e['entity'],e['value']) for e in entities])

[{u'extractor': u'ner_mitie', u'end': 11, u'start': 5, u'value': u'Horror', u'entity': u'dbpprop:genre'}]
Intent: moviesOfGenre
Intent confidence: 0.595925237943
Entities: (dbpprop:genre,Horror)


In [17]:
from backend.contextbased.default_intents import *

new_context = Context(moviesOfGenre(entities, old_context))

query, select_columns = generate_sparql_from_context(new_context, prefixes=False)
print query

a
a
 
 SELECT  ?Film_1_dbpprop_name WHERE { 
?Film1 rdf:type dbp-owl:Film.
?Film1 dbpprop:name ?Film_1_dbpprop_name.
?Film1 ns:release ?Release1.
?Film1 dbpprop:genre "Horror".
?Release1 rdf:type ns:Release.
?Release1 dbpprop:releaseDate 2016.
?Release1 dbpprop:releaseLocation "USA". 
 } LIMIT 10


### Test 3
User: *what is nemo about?*

Bot: plot of inception

User: *who is starring in?*

In [29]:
text1 = "what is django unchained about?"
intent, intent_confidence, entities, properties, literal_objects, ranges = ch.rasa.extract_from_text(text1)

old_context = Context(entities)

query, select_columns = generate_sparql_from_context(old_context, prefixes=False)
print query

Intent: plotOf
Intent confidence 0.000000
Result entities: [{u'extractor': u'ner_mitie', u'end': 14, u'start': 8, u'value': u'django', u'entity': u'dbp-owl:Film'}, {u'extractor': u'ner_mitie', u'end': 24, u'start': 15, u'value': u'unchained', u'entity': u'dbp-owl:Film'}]
 
 SELECT  WHERE { 
?Film1 rdf:type dbp-owl:Film.
?Film1 dbpprop:name "django".
?Film1 rdf:type dbp-owl:Film.
?Film1 dbpprop:name "unchained". 
 } LIMIT 10


In [30]:
text2 = "who is starring in?"

result = rasa.query(text2)

intent = result['intent']['name']
intent_confidence = result['intent']['confidence']
entities = result['entities']
print entities

print 'Intent: %s' % intent
print 'Intent confidence: %s' % intent_confidence
print 'Entities: %s' % ' '.join(['(%s,%s)' % (e['entity'],e['value']) for e in entities])

[]
Intent: starringIn
Intent confidence: 0.822029720187
Entities: 


In [31]:
from backend.contextbased.default_intents import *

new_context = Context(starringIn(entities, old_context))

query, select_columns = generate_sparql_from_context(new_context, prefixes=False)
print query

 
 SELECT  ?Actor_1_foaf_name ?Actor_1_foaf_familyName WHERE { 
?Film1 rdf:type dbp-owl:Film.
?Film1 dbpprop:name "django".
?Film1 dbpprop:starring ?Actor1.
?Actor1 rdf:type dbp-owl:Actor.
?Actor1 foaf:name ?Actor_1_foaf_name.
?Actor1 foaf:familyName ?Actor_1_foaf_familyName. 
 } LIMIT 10
